In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import datasets
train=pd.read_csv("./imdb/labeledTrainData.tsv", delimiter="\t", quoting=3)
test=pd.read_csv("./imdb/testData.tsv", delimiter="\t", quoting=3)

#划分验证集和训练集
train,val = train_test_split(train,test_size=0.2)

#构建字典
train_dict = {'label': train["sentiment"].astype(int), 'text': train['review']}
val_dict = {'label': val["sentiment"].astype(int), 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)

print(val_dataset)


Dataset({
    features: ['label', 'text'],
    num_rows: 5000
})


In [6]:
from transformers import DebertaV2Tokenizer, AutoModelForSequenceClassification
import os
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split

train = pd.read_csv("/root/imdb/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/root/imdb/testData.tsv", header=0, delimiter="\t", quoting=3)

train, val = train_test_split(train, test_size=.2)

train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)

local_dir = "/root/autodl-tmp/models/deberta-v2-xxlarge/"

tokenizer = DebertaV2Tokenizer.from_pretrained(
    local_dir,
    use_fast=False  # 只在 tokenizer 这儿有效
)

model = AutoModelForSequenceClassification.from_pretrained(
    local_dir,
    num_labels=2   # 二分类头大小，模型里会自动创建一个新的 classification head
)

def preprocess_f(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding=False,
        max_length=497
    )

tokenized_train = train_dataset.map(preprocess_f, batched=True)
tokenized_val   = val_dataset.map(preprocess_f, batched=True)
tokenized_test  = test_dataset.map(preprocess_f, batched=True)

print(tokenized_train)


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/models/deberta-v2-xxlarge/ and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})


In [5]:
!pip install --upgrade torch torchvision

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 2.1 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 23.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 21.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 17.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.7 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from transformers import DebertaV2Tokenizer, AutoModelForSequenceClassification
import os
from requests.exceptions import ConnectionError

# —— 1. 全部用 SOCKS5h 代理 —— 
socks_proxy = "socks5h://127.0.0.1:7891"
for v in ("http_proxy","https_proxy","all_proxy",
          "HTTP_PROXY","HTTPS_PROXY","ALL_PROXY"):
    os.environ[v] = socks_proxy

# HF Hub 专用代理（无所谓 http/https，统一用 socks）
os.environ["HF_HUB_HTTP_PROXY"]  = socks_proxy
os.environ["HF_HUB_HTTPS_PROXY"] = socks_proxy

proxies = {
    "http":  socks_proxy,
    "https": socks_proxy,
}

model_id  = "microsoft/deberta-v2-xxlarge"
local_dir = "/workspace/models/deberta-v2-xxlarge"  # 如果你已有本地仓库

# —— 2. 尝试用 SOCKS5 下载，失败再退本地 —— 
try:
    print(">> 正在通过 SOCKS5 代理下载模型……")
    tokenizer = DebertaV2Tokenizer.from_pretrained(
        model_id,
        proxies=proxies,
        trust_remote_code=False
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        num_labels=2,
        use_fast=False,
        proxies=proxies
    )
except (ConnectionError, OSError) as e:
    print(f">> 下载仍然失败 ({e})，改为本地加载：{local_dir}")
    tokenizer = DebertaV2Tokenizer.from_pretrained(local_dir)
    model     = AutoModelForSequenceClassification.from_pretrained(
                    local_dir,
                    num_labels=2,
                    use_fast=False
                )

# —— 3. 数据预处理 —— 
def preprocess_f(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding=False,
        max_length=497
    )

tokenized_train = train_dataset.map(preprocess_f, batched=True)
tokenized_val   = val_dataset.map(preprocess_f, batched=True)
tokenized_test  = test_dataset.map(preprocess_f, batched=True)

print(tokenized_train)


>> 正在通过 SOCKS5 代理下载模型……
>> 下载仍然失败 ((ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 7b3f7847-fe09-4d24-9d4f-0872dd5f31c8)'))，改为本地加载：/workspace/models/deberta-v2-xxlarge


OSError: Can't load tokenizer for '/workspace/models/deberta-v2-xxlarge'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/workspace/models/deberta-v2-xxlarge' is the correct path to a directory containing all relevant files for a DebertaV2Tokenizer tokenizer.

In [4]:
!git clone https://huggingface.co/microsoft/deberta-v2-xxlarge

Cloning into 'deberta-v2-xxlarge'...
fatal: unable to access 'https://huggingface.co/microsoft/deberta-v2-xxlarge/': gnutls_handshake() failed: Error in the pull function.


In [7]:
import os

http_proxy  = "http://127.0.0.1:7890"
https_proxy = "http://127.0.0.1:7890"
socks_proxy = "socks5h://127.0.0.1:7891"

# 同时设置 HTTP 和 SOCKS5，大小写都打齐
for k,v in {
    "http_proxy":  http_proxy,
    "https_proxy": https_proxy,
    "HTTP_PROXY":  http_proxy,
    "HTTPS_PROXY": https_proxy,
    "all_proxy":   socks_proxy,
    "ALL_PROXY":   socks_proxy,
}.items():
    os.environ[k] = v

# Transformers Hub 专用 proxy 环境变量
os.environ["HF_HUB_HTTP_PROXY"]  = http_proxy
os.environ["HF_HUB_HTTPS_PROXY"] = https_proxy

print(os.environ.get("http_proxy"), os.environ.get("HF_HUB_HTTP_PROXY"))


http://127.0.0.1:7890 http://127.0.0.1:7890


In [2]:
!curl -v -x http://127.0.0.1:7890 https://huggingface.co

*   Trying 127.0.0.1:7890...
* Connected to (nil) (127.0.0.1) port 7890 (#0)
* allocate connect buffer!
* Establish HTTP proxy tunnel to huggingface.co:443
> CONNECT huggingface.co:443 HTTP/1.1
> Host: huggingface.co:443
> User-Agent: curl/7.81.0
> Proxy-Connection: Keep-Alive
> 
< HTTP/1.1 200 Connection established
< 
* Proxy replied 200 to CONNECT request
* CONNECT phase completed!
* ALPN, offering h2
* ALPN, offering http/1.1
*  CAfile: /etc/ssl/certs/ca-certificates.crt
*  CApath: /etc/ssl/certs
* TLSv1.0 (OUT), TLS header, Certificate Status (22):
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
* OpenSSL SSL_connect: Connection reset by peer in connection to huggingface.co:443 
* Closing connection 0
* TLSv1.0 (OUT), TLS header, Unknown (21):
* TLSv1.3 (OUT), TLS alert, decode error (562):
curl: (35) OpenSSL SSL_connect: Connection reset by peer in connection to huggingface.co:443 


In [ ]:
!curl http://127.0.0.1:9090/proxies/Proxy

In [7]:
!curl -X PUT -H "Content-Type: application/json" -d '{"name": "GLaDOS-US-02"}' http://127.0.0.1:9090/proxies/Proxy

In [5]:
!curl -x socks5h://127.0.0.1:7890 https://huggingface.co

curl: (28) Connection timed out after 300498 milliseconds


In [2]:
!export http_proxy="socks5h://127.0.0.1:7891"
!export https_proxy="socks5h://127.0.0.1:7891"
!export all_proxy="socks5h://127.0.0.1:7891"

# 小写/大写都设置一遍保险
!export HTTP_PROXY=$http_proxy
!export HTTPS_PROXY=$https_proxy
!export ALL_PROXY=$all_proxy


In [4]:
import os

# 把 socks5 代理同时赋给 大小写 + all_proxy
proxy_url = "socks5h://127.0.0.1:7891"
for v in ("http_proxy","https_proxy","all_proxy",
          "HTTP_PROXY","HTTPS_PROXY","ALL_PROXY"):
    os.environ[v] = proxy_url

# 验证一下
print("http_proxy =", os.environ.get("http_proxy"))
print("HTTPS_PROXY =", os.environ.get("HTTPS_PROXY"))

http_proxy = socks5h://127.0.0.1:7891
HTTPS_PROXY = socks5h://127.0.0.1:7891


In [10]:
#设置参数
from peft import PromptTuningConfig, get_peft_model
peft_config = PromptTuningConfig(
    task_type="SEQ_CLS",  
    num_virtual_tokens=15,  
    token_dim=model.config.hidden_size,  
    num_transformer_submodules=1
)
model = get_peft_model(model, peft_config)

In [15]:
#设置训练参数
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./prompt_tuning_imdb",
    learning_rate=3e-2,  # Prompt Tuning 通常需要更大的学习率
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    report_to="none"
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
#训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

In [ ]:
#预测
prediction_outputs = trainer.predict(tokenized_test)
test_pred = np.argmax(prediction_outputs.predictions,axis=-1).flatten()
test = dataset["test"]

result_output = pd.DataFrame(data={"id":test["id"],"sentiment":test_pred})
result_output.to_csv("./result/imdb_deberta_prompt.csv")
print("Saved!")

In [6]:
from transformers import DebertaV2Tokenizer, AutoModelForSequenceClassification
import os
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split
from peft import PromptTuningConfig, get_peft_model
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

train = pd.read_csv("/root/imdb/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/root/imdb/testData.tsv", header=0, delimiter="\t", quoting=3)

train, val = train_test_split(train, test_size=.2)

train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)

local_dir = "/root/autodl-tmp/models/deberta-v2-xxlarge/"

tokenizer = DebertaV2Tokenizer.from_pretrained(
    local_dir,
    use_fast=False  # 只在 tokenizer 这儿有效
)

model = AutoModelForSequenceClassification.from_pretrained(
    local_dir,
    num_labels=2   # 二分类头大小，模型里会自动创建一个新的 classification head
)

def preprocess_f(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding=False,
        max_length=497
    )

tokenized_train = train_dataset.map(preprocess_f, batched=True)
tokenized_val   = val_dataset.map(preprocess_f, batched=True)
tokenized_test  = test_dataset.map(preprocess_f, batched=True)

print(tokenized_train)

#设置参数
peft_config = PromptTuningConfig(
    task_type="SEQ_CLS",  
    num_virtual_tokens=15,  
    token_dim=model.config.hidden_size,  
    num_transformer_submodules=1
)
model = get_peft_model(model, peft_config)

#设置训练参数
training_args = TrainingArguments(
    output_dir="./prompt_tuning_imdb",
    learning_rate=3e-2,  # Prompt Tuning 通常需要更大的学习率
    per_device_train_batch_size=1,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    report_to="none",
    # gradient_checkpointing=True
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
#训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
# #预测
# prediction_outputs = trainer.predict(tokenized_test)
# test_pred = np.argmax(prediction_outputs.predictions,axis=-1).flatten()
# test = dataset["test"]

# result_output = pd.DataFrame(data={"id":test["id"],"sentiment":test_pred})
# result_output.to_csv("/root/code/imdb_deberta_prompt.csv")
# print("Saved!")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/models/deberta-v2-xxlarge/ and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})


/tmp/ipykernel_1676/172154668.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,0.699900,0.694562
2,0.704900,0.691791
3,0.697000,0.713389
4,0.691600,0.698095


NameError: name 'np' is not defined

In [7]:
import numpy as np
#预测
prediction_outputs = trainer.predict(tokenized_test)
test_pred = np.argmax(prediction_outputs.predictions,axis=-1).flatten()

NameError: name 'dataset' is not defined

In [9]:
from transformers import DebertaV2Tokenizer, AutoModelForSequenceClassification
import os
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split
from peft import PromptTuningConfig, get_peft_model
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

# test_pred = np.argmax(prediction_outputs.predictions, axis=-1).flatten()

# 使用原始 test DataFrame 的 id 列
test = pd.read_csv("/root/imdb/testData.tsv", header=0, delimiter="\t", quoting=3)
result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/root/code/imdb_deberta_prompt.csv", index=False)
print("Saved!")

Saved!
